In [11]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from xml.etree import ElementTree as ET

SERVICE_KEY = "7vMdnzTpnFnBO5wPN3LkHyPgPNFu3A/w/+RH8EJw3ihZfuhA5UiMx4x/PYl1qjlCx1VAzTL+i2GJXf1c/oHfyg=="

# 주소 파싱
def parse_address(address):
    tokens = address.strip().split()
    gu_index = next(i for i, t in enumerate(tokens) if t.endswith("구") or t.endswith("군"))
    region_name = " ".join(tokens[:gu_index + 1])
    dong = tokens[gu_index + 1]
    jibun = tokens[gu_index + 2]
    return region_name, dong, jibun

# 법정동 코드 조회
def get_region_prefix(region_name):
    url = "http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList"
    params = {
        'ServiceKey': SERVICE_KEY,
        'type': 'json',
        'pageNo': '1',
        'numOfRows': '1000',
        'flag': 'Y',
        'locatadd_nm': region_name
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        stanregin = data.get('StanReginCd', [])
        for item in stanregin:
            if 'row' in item:
                for entry in item['row']:
                    if entry.get('locatadd_nm') == region_name:
                        return entry['region_cd'][:5]
    return None

# 건물유형별 API URL 매핑
def get_trade_api_url(building_type):
    mapping = {
        "아파트": "AptTrade",
        "오피스텔": "OffiTrade",
        "다세대": "RHTrade",
        "연립": "RHTrade"
    }
    suffix = mapping.get(building_type)
    if suffix:
        return f"https://apis.data.go.kr/1613000/RTMSDataSvc{suffix}/getRTMSDataSvc{suffix}"
    return None

# 월 리스트 생성
def get_month_list(months=60):
    today = datetime.today()
    return [(today - timedelta(days=30 * i)).strftime("%Y%m") for i in range(months)]

# ✅ 통합 거래 내역 조회 함수
def get_trade_history(address, building_type="아파트"):
    region, dong, jibun = parse_address(address)
    lawd_cd = get_region_prefix(region)
    if not lawd_cd:
        print("❌ 법정동 코드 조회 실패")
        return pd.DataFrame()

    API_URL = get_trade_api_url(building_type)
    if not API_URL:
        print("❌ 건물 유형이 잘못되었습니다.")
        return pd.DataFrame()

    all_rows = []

    for yyyymm in get_month_list():
        page = 1

        while True:
            params = {
                "serviceKey": SERVICE_KEY,
                "LAWD_CD": lawd_cd,
                "DEAL_YMD": yyyymm,
                "numOfRows": "1000",
                "pageNo": str(page)
            }

            try:
                response = requests.get(API_URL, params=params)
                root = ET.fromstring(response.content)

                items = list(root.iter("item"))
                if not items:
                    break  # 이 페이지에 데이터가 없으면 끝

                for item in items:
                    item_dong = item.findtext("umdNm", "").strip()
                    item_jibun = item.findtext("jibun", "").strip()
                    if item_dong != dong or item_jibun != jibun:
                        continue

                    year = item.findtext("dealYear")
                    month = item.findtext("dealMonth")
                    day = item.findtext("dealDay")

                    if not (year and month and day):
                        continue

                    deal_amount = int(item.findtext("dealAmount", "0").replace(",", ""))
                    area = float(item.findtext("excluUseAr", "0"))

                    row = {
                        "건물유형": building_type,
                        "법정동": item_dong,
                        "지번": item_jibun,
                        "거래금액(만원)": deal_amount,
                        "전용면적(㎡)": area,
                        "㎡당가격(만원)": round(deal_amount / area, 2) if area else None,
                        "계약일": f"{year}-{month.zfill(2)}-{day.zfill(2)}",
                        "층": item.findtext("floor", "").strip(),
                        "단지명": item.findtext("aptNm", "").strip() if building_type == "아파트" else item.findtext("buildingName", "").strip()
                    }

                    all_rows.append(row)

                if len(items) < 1000:
                    break  # 마지막 페이지 도달
                page += 1

            except Exception as e:
                print(f"❌ 오류 ({yyyymm} page {page}): {e}")
                break

    return pd.DataFrame(all_rows)

# ✅ 실행 예시
address = "서울특별시 관악구 봉천동 148-149"  # 오피스텔 주소 예시
building_type = "다세대"  # 또는 "아파트", "다세대"

df = get_trade_history(address, building_type)

if not df.empty:
    print(f"\n✅ 최근 5년간 거래 내역 ({address}, {building_type}):")
    print(df.sort_values("계약일", ascending=False))
else:
    print(f"\n⚠️ 거래 내역이 없습니다: {address}, {building_type}")


✅ 최근 5년간 거래 내역 (서울특별시 관악구 봉천동 148-149, 다세대):
  건물유형  법정동       지번  거래금액(만원)  전용면적(㎡)  ㎡당가격(만원)         계약일  층 단지명
0  다세대  봉천동  148-149     31000    73.55    421.48  2024-09-08  2    
1  다세대  봉천동  148-149     33000    64.24    513.70  2023-09-17  3    
2  다세대  봉천동  148-149     29000    64.24    451.43  2023-08-31  2    
3  다세대  봉천동  148-149     32500    64.24    505.92  2023-08-08  3    
4  다세대  봉천동  148-149     24000    73.55    326.31  2021-05-27  1    
5  다세대  봉천동  148-149     24500    64.24    381.38  2020-09-11  3    


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from xml.etree import ElementTree as ET

def parse_address(address):
    tokens = address.strip().split()
    gu_index = next(i for i, t in enumerate(tokens) if t.endswith("구") or t.endswith("군"))
    region_name = " ".join(tokens[:gu_index + 1])
    dong = tokens[gu_index + 1]
    jibun = tokens[gu_index + 2]
    return region_name, dong, jibun

def get_region_prefix(region_name):
    url = "http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList"
    params = {
        'ServiceKey': "7vMdnzTpnFnBO5wPN3LkHyPgPNFu3A/w/+RH8EJw3ihZfuhA5UiMx4x/PYl1qjlCx1VAzTL+i2GJXf1c/oHfyg==",
        'type': 'json',
        'pageNo': '1',
        'numOfRows': '1000',
        'flag': 'Y',
        'locatadd_nm': region_name
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        stanregin = data.get('StanReginCd', [])
        for item in stanregin:
            if 'row' in item:
                for entry in item['row']:
                    if entry.get('locatadd_nm') == region_name:
                        return entry['region_cd'][:5]
    return None

def get_api_url(building_type):
    mapping = {
        "아파트": "RTMSDataSvcAptRent",
        "오피스텔": "RTMSDataSvcOffiRent",
        "다세대": "RTMSDataSvcRHRent",
        "연립": "RTMSDataSvcRHRent"
    }
    api_name = mapping.get(building_type)
    if api_name:
        return f"http://apis.data.go.kr/1613000/{api_name}/get{api_name}"
    else:
        print(f"❌ 지원하지 않는 건물 유형: {building_type}")
        return None

def get_jeonse_history(address, building_type):
    region, dong, jibun = parse_address(address)
    lawd_cd = get_region_prefix(region)
    if not lawd_cd:
        print("❌ 법정동 코드 조회 실패")
        return pd.DataFrame()

    api_url = get_api_url(building_type)
    if not api_url:
        return pd.DataFrame()

    all_rows = []

    for yyyymm in get_month_list():
        page = 1
        while True:
            params = {
                "serviceKey": "7vMdnzTpnFnBO5wPN3LkHyPgPNFu3A/w/+RH8EJw3ihZfuhA5UiMx4x/PYl1qjlCx1VAzTL+i2GJXf1c/oHfyg==",
                "LAWD_CD": lawd_cd,
                "DEAL_YMD": yyyymm,
                "numOfRows": "1000",
                "pageNo": str(page)
            }
            try:
                response = requests.get(api_url, params=params)
                root = ET.fromstring(response.content)
                items = list(root.iter("item"))
                if not items:
                    break
                for item in items:
                    item_dong = item.findtext("umdNm", "").strip()
                    item_jibun = item.findtext("jibun", "").strip()
                    if item_dong != dong or item_jibun != jibun:
                        continue

                    deposit = int(item.findtext("deposit", "0").replace(",", "").strip())
                    monthly_rent = int(item.findtext("monthlyRent", "0").replace(",", "").strip())

                    area = float(item.findtext("excluUseAr", "0").strip())
                    row = {
                        "법정동": item_dong,
                        "지번": item_jibun,
                        "전용면적(㎡)": area,
                        "보증금(만원)": deposit,
                        "월세(만원)": monthly_rent,
                        "계약년도": item.findtext("dealYear", "").strip(),
                        "계약월": item.findtext("dealMonth", "").strip(),
                        "계약일": item.findtext("dealDay", "").strip(),
                        "㎡당보증금(만원)": round(deposit / area, 2) if area else None
                    }
                    all_rows.append(row)

                if len(items) < 1000:
                    break
                page += 1

            except Exception as e:
                print(f"❌ 오류 ({yyyymm} page {page}): {e}")
                break

    df = pd.DataFrame(all_rows)
    if not df.empty:
        df["계약일"] = pd.to_datetime(
            df["계약년도"] + "-" + df["계약월"].str.zfill(2) + "-" + df["계약일"].str.zfill(2),
            errors='coerce'
        )
    return df

def get_month_list(months=60):
    today = datetime.today()
    return [(today - timedelta(days=30 * i)).strftime("%Y%m") for i in range(months)]

# ✅ 실행 예시
address = "서울특별시 관악구 봉천동 1708"
building_type = "아파트"  # "아파트", "오피스텔", "다세대" 선택 가능

df_jeonse = get_jeonse_history(address, building_type)
print(df_jeonse.head(20))